# 🏠 Ev Fiyat Tahmin Projesi - Feature Engineering ve Veri Ön İşleme

Bu notebook, eksik değer doldurma ve basit özellik türetme adımlarını içerir.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)
print("✅ Kütüphaneler yüklendi")


In [ ]:
train = pd.read_csv('datasets/housePrice/house_price_train.csv')
test = pd.read_csv('datasets/housePrice/house_price_test.csv')
df = pd.concat([train, test], ignore_index=True)
print(df.shape)
df.head(2)


In [ ]:
# Eksik değer doldurma: sayısal = median, kategorik = mode
num_cols = df.select_dtypes(include=[np.number]).columns
cat_cols = df.select_dtypes(include=['object']).columns

if len(num_cols):
    df[num_cols] = SimpleImputer(strategy='median').fit_transform(df[num_cols])
if len(cat_cols):
    df[cat_cols] = SimpleImputer(strategy='most_frequent').fit_transform(df[cat_cols])

print("Eksik değer doldurma tamamlandı")
df.isnull().sum().sum()


In [ ]:
# Basit özellikler
if {'1stFlrSF','2ndFlrSF','TotalBsmtSF'}.issubset(df.columns):
    df['TotalSF'] = df['1stFlrSF'] + df['2ndFlrSF'] + df['TotalBsmtSF']

print("Yeni sütunlar:", [c for c in df.columns if c.endswith('SF')][-5:])
df[['TotalSF']].head() if 'TotalSF' in df.columns else df.head(1)


In [ ]:
# Notebook çıktısı olarak sadece df şekli gösterelim (kaydetme opsiyonel)
print("Güncel şekil:", df.shape)
